# Graph Module Verification

This notebook tests the `src.graph` module which implements:
- **Self-CRAG workflow** with retry loop
- **Graph nodes** (rewrite, retrieve, grade, generate, hallucination check, fallback)
- **Conditional edges** (decide_to_generate, decide_to_final)

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from src.graph import (
    build_graph, MAX_RETRIES,
    rewrite_node, retrieve_node, grade_documents_node,
    generate_node, hallucination_check_node, fallback_node,
    decide_to_generate, decide_to_final
)

print(f"MAX_RETRIES = {MAX_RETRIES}")

## Step 1: Build Graph

Compile the LangGraph workflow.

In [ ]:
app = build_graph()

print(f"Graph Built:")
print(f"   • Type: {type(app).__name__}")
print(f"   • MAX_RETRIES: {MAX_RETRIES}")

print("\nGraph compiled successfully!")

## Step 2: Test Individual Nodes

Each node receives state and returns updates.

In [ ]:
# Mock state
mock_state = {
    "messages": [],
    "question": "What is the capital of France?",
    "standalone_question": "",
    "documents": [],
    "generation": "",
    "documents_relevant": "no",
    "is_hallucination": "no",
    "retry_count": 0
}

print("Node Outputs:")
print(f"   • rewrite_node: {rewrite_node(mock_state)}")
print(f"   • retrieve_node: {retrieve_node(mock_state)}")
print(f"   • grade_documents_node: {grade_documents_node(mock_state)}")
print(f"   • generate_node: {generate_node(mock_state)}")
print(f"   • hallucination_check_node: {hallucination_check_node(mock_state)}")
print(f"   • fallback_node: {fallback_node(mock_state)}")

print("\nAll nodes work correctly!")

## Step 3: Test Conditional Edges

These functions determine which path the graph takes.

In [ ]:
print("decide_to_generate (after grading docs):")
print(f"   • docs_relevant='yes' → {decide_to_generate({'documents_relevant': 'yes'})}")
print(f"   • docs_relevant='no'  → {decide_to_generate({'documents_relevant': 'no'})}")

print("\ndecide_to_final (after hallucination check):")
print(f"   • hallucination='no'           → {decide_to_final({'is_hallucination': 'no', 'retry_count': 0})}")
print(f"   • hallucination='yes', retry=0 → {decide_to_final({'is_hallucination': 'yes', 'retry_count': 0})}")
print(f"   • hallucination='yes', retry={MAX_RETRIES} → {decide_to_final({'is_hallucination': 'yes', 'retry_count': MAX_RETRIES})}")

print("\nAll conditional edges work correctly!")

## Step 4: Full Graph Execution

Run the complete workflow (with dummy nodes).

In [ ]:
print("Executing graph...")

result = app.invoke({"question": "What is the capital of France?"})

print("\nFinal State:")
print(f"   • standalone_question: {result.get('standalone_question')}")
print(f"   • documents: {len(result.get('documents', []))} docs")
print(f"   • documents_relevant: {result.get('documents_relevant')}")
print(f"   • generation: '{result.get('generation')}'")
print(f"   • is_hallucination: {result.get('is_hallucination')}")
print(f"   • retry_count: {result.get('retry_count')}")

print("\nGraph executed successfully!")